# **Classification Food/NoFood**<br/>
**Master's Degree in Data Science (A.Y. 2023/2024)**<br/>
**University of Milano - Bicocca**<br/>

Vittorio Haardt, Luca Porcelli

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Extraction Train

In [ ]:
!unzip "/content/drive/MyDrive/VIPM/Dataset/archive.zip" -d train

# Installing packages and loading libraries

In [ ]:
pip install efficientnet

In [ ]:
import tensorflow as tf
import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, BatchNormalization, Dropout, Dense
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras import layers, models
from efficientnet.keras import EfficientNetB0
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, EarlyStopping, ModelCheckpoint

# Data Augmentation

In [ ]:
# Creating an image generator with various transformations for data augmentation
datagen = ImageDataGenerator(
    rotation_range=40,          # Maximum rotation angle in degrees
    zoom_range=0.2,             # Random zoom
    horizontal_flip=True,       # Randomly flip images horizontally
    vertical_flip=True,         # Randomly flip images vertically
    brightness_range=[0.5, 1.5] # Range of brightness variation
)

# Train

In [ ]:
# Create a dataset of images from the specified directory
train = datagen.flow_from_directory(
    "/content/train/archive/training",    # Path to the directory containing training images
    classes=None,                         # Use default class directories
    class_mode="categorical",             # Specify the encoding mode for labels
    color_mode="rgb",                     # RGB format for images
    batch_size=64,                        # Batch size
    target_size=(224, 224),               # Set the dimensions of the images to 224x224 pixels
    shuffle=True,                         # Shuffle images within the dataset
    seed=777                              # Seed for reproducibility
)

Found 3000 images belonging to 2 classes.


# Validation

In [ ]:
# Create a dataset of images from the specified directory for validation
val = datagen.flow_from_directory(
    "/content/train/archive/validation",  # Path to the directory containing validation images
    classes=None,                         # Use default class directories
    class_mode="categorical",             # Specify the encoding mode for labels
    color_mode="rgb",                     # RGB format for images
    batch_size=64,                        # Batch size
    target_size=(224, 224),               # Set the dimensions of the images to 224x224 pixels
    shuffle=True,                         # Shuffle images within the dataset
    seed=777                              # Seed for reproducibility
)

Found 1000 images belonging to 2 classes.


# EfficientNetB0

## Structure

In [ ]:
# Creating the base EfficientNetB0 model pretrained on ImageNet
base_eff_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
trainable_layer = False  # Setting the layers of the base model as non-trainable

# Explicitly creating the input layer
inputs = keras.Input(shape=(224, 224, 3))
x = inputs

# Preprocessing the input using the EfficientNetB0 preprocessing function
x = keras.applications.efficientnet.preprocess_input(x)

# Passing the preprocessed input through the base model
x = base_eff_model(x)

# GlobalAveragePooling2D to reduce spatial dimensions
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.3)(x)  # Dropout for regularization

# Adding a dense layer for final classification with L2 regularization
outputs = keras.layers.Dense(2, activation='softmax', kernel_regularizer=keras.regularizers.l2(0.01))(x)

# Creating the specialized EfficientNet model
Efficientnet = keras.Model(inputs=inputs, outputs=outputs)

16804768/16804768 [==============================] - 0s 0us/step


In [ ]:
# Definition of a function for learning rate scheduling
def decay_schedule(epoch, lr):
    # Check if the epoch is a multiple of 5 and if the epoch is not zero
    if (epoch % 5 == 0) and (epoch != 0):
        # Reduce the learning rate by 20%
        lr = lr * 0.8
    # Return the new learning rate value
    return lr

# Creating a LearningRateScheduler object using the learning rate schedule function
lr_scheduler = LearningRateScheduler(decay_schedule)

# Early stopping callback
early_stop = EarlyStopping(
    monitor='val_loss',               # Monitors validation loss
    patience=5,                       # Number of epochs with no improvement before stopping
    verbose=1,                        # Prints messages about early stopping
    restore_best_weights=True         # Restores model weights from the epoch with the best value of the monitored quantity
)

## Training

In [ ]:
# Compiling the model for training
Efficientnet.compile(
    loss='categorical_crossentropy',                                     # Categorical crossentropy loss for multi-class classification
    optimizer=Adam(0.0001),                                              # Adam optimizer with a learning rate of 0.0001
    metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=5)]  # Metrics for evaluation during training
)

# Training the model
history = Efficientnet.fit(
    train,                                # Training data generator
    epochs=50,                            # Number of training epochs
    validation_data=val,                  # Validation data generator
    callbacks=[lr_scheduler, early_stop]  # Callbacks for learning rate scheduling and early stopping
)

Epoch 1/50
47/47 [==============================] - 120s 2s/step - loss: 0.3510 - accuracy: 0.8773 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.4685 - val_accuracy: 0.7920 - val_top_k_categorical_accuracy: 1.0000 - lr: 1.0000e-04
Epoch 2/50
47/47 [==============================] - 79s 2s/step - loss: 0.1445 - accuracy: 0.9723 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.2568 - val_accuracy: 0.9240 - val_top_k_categorical_accuracy: 1.0000 - lr: 1.0000e-04
Epoch 3/50
47/47 [==============================] - 78s 2s/step - loss: 0.1027 - accuracy: 0.9817 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.1485 - val_accuracy: 0.9580 - val_top_k_categorical_accuracy: 1.0000 - lr: 1.0000e-04
Epoch 4/50
47/47 [==============================] - 79s 2s/step - loss: 0.0805 - accuracy: 0.9877 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.1167 - val_accuracy: 0.9770 - val_top_k_categorical_accuracy: 1.0000 - lr: 1.0000e-04
Epoch 5/50
47/47 [==============================] - 74s 2s/

## Saving model

In [ ]:
# Saving the trained model
Efficientnet.save('/content/drive/MyDrive/VIPM/Models/Model_PDF.h5')
Efficientnet.save('/content/drive/MyDrive/VIPM/Models/Model_PDF.Keras')

# Evaluation

In [ ]:
# Creating a test dataset using tf.keras.utils.image_dataset_from_directory
test = tf.keras.utils.image_dataset_from_directory(
    "/content/train/archive/evaluation",     # Path to the directory containing evaluation images
    labels="inferred",                       # Infers labels from the directory structure
    label_mode="categorical",                # Specify the encoding mode for labels
    color_mode="rgb",                        # RGB format for images
    batch_size=64,                           # Batch size
    image_size=(224, 224),                   # Set the dimensions of the images to 224x224 pixels
    shuffle=True                             # Shuffle images within the dataset
)

Found 1000 files belonging to 2 classes.


In [ ]:
Efficientnet.evaluate(test)

16/16 [==============================] - 6s 194ms/step - loss: 0.0586 - accuracy: 0.9890 - top_k_categorical_accuracy: 1.0000


[0.05862918123602867, 0.9890000224113464, 1.0]

**Reference**
- Dataset: [Food5k](https://www.kaggle.com/datasets/trolukovich/food5k-image-dataset)